# Acoustic Forward Simulation

In this notebook we will go through a simple example of setting up a purely acoustic forward simulation in PestoSeis for a 2D overthrust model.

# Preamble

In [ ]:
import pestoseis
import numpy as np
import matplotlib.pyplot as plt
import h5py
from scipy.interpolate import RectBivariateSpline

# Model Interpolation

First, we will import the HDF5 (`.h5`) model which contains the following keys:
- `vp`: P-wave velocity model (in units of \[m/s\])
- `ijsrcs`: Indices of the source positions
- `ijrecs`: Indices of the receiver positions

In [ ]:
data_path = "overthrust_model.h5"
with h5py.File(str(data_path), "r") as f:
    vp = np.array(f["vp"])
    sources = np.array(f["ijsrcs"], dtype=int)
    receivers = np.array(f["ijrecs"], dtype=int)

We can plot the velocity model to verify that the data has been imported correctly.

In [ ]:
plt.figure(figsize=[10, 16])
im = plt.imshow(vp.T, origin="upper", cmap="jet")
plt.plot(sources[0, :], sources[1, :], "*r")
plt.plot(receivers[0, :], receivers[1, :], ".c")
plt.xlabel("x-Position")
plt.ylabel("z-Position")
plt.title("Overthrust P-Wave Velocity Model [m/s]")
plt.colorbar(im, fraction=0.046*(10/16), pad=0.04)
plt.show()

Notice that the P-wave velocity model is currently fairly coarse (ie. there are relatively few samples in the x- and z-directions).  This discretisation is currently too coarse for the desired source frequency we want to use for this example (50 Hz), so we need to interpolate this coarser model onto a finer grid.  A fairly simple way of accomplishing this is to use the `RectBivariateSpline` tool from the `scipy` package.

We can set a desired scaling factor (see `scale`) which will scale the number of samples in each of the x- and z-directions.

In [ ]:
scale = 2

interp_fun = RectBivariateSpline(np.linspace(0, vp.shape[0], vp.shape[0]), np.linspace(0, vp.shape[1], vp.shape[1]), vp)
vp = interp_fun(np.linspace(0, vp.shape[0], vp.shape[0]*scale), np.linspace(0, vp.shape[1], vp.shape[1]*scale))

sources *= scale
receivers *= scale

plt.figure(figsize=[10, 16])
plt.imshow(vp.T, origin="upper", cmap="jet")
plt.plot(sources[0, :], sources[1, :], "*r")
plt.plot(receivers[0, :], receivers[1, :], ".c")
plt.xlabel("x-Position")
plt.ylabel("z-Position")
plt.title("Overthrust P-Wave Velocity Model [m/s]")
plt.show()

# Simulation Setup

Next, we can set up the simulation itself.  For this example, we will be explicitly defining the following:
- Time step
- Number of time steps
- Source time function characteristics (Ricker wavelet)
- Free surface boundary conditions at the surface of the domain
- PMLs on all other sides

In [ ]:
# Temporal discretization
nt = 2000
dt = 0.0005
t = np.arange(0.0, nt*dt, dt)

# Spatial discretization
nx = vp.shape[0]
nz = vp.shape[1]
dh = 10 / scale

# Sources
t0 = 0.03
f0 = 25.0
ijsrc = sources[:, 2]

# Source time function
sourcetf = pestoseis.seismicwaves2d.sourcetimefuncs.rickersource( t, t0, f0 )

# Receivers
nrec = receivers.shape[1]
recpos = receivers.T * dh

print(("Receiver positions:\n{}".format(recpos)))

# Initiallize input parameter dictionary
inpar = {}
inpar["ntimesteps"] = nt
inpar["nx"] = nx
inpar["nz"] = nz
inpar["dt"] = dt
inpar["dh"] = dh
inpar["savesnapshot"] = True
inpar["snapevery"] = 50
inpar["freesurface"] = True
inpar["boundcond"] = "PML" #"GaussTap", "PML", "GaussTap","ReflBou"

# Running the Simulation

Now that we have set up the forward simulation, we can run this using `solveacoustic2D` in `pestoseis`.

In [ ]:
seism, psave = pestoseis.seismicwaves2d.acousticwaveprop2D.solveacoustic2D(inpar, ijsrc, vp, sourcetf, f0, recpos)

# Plotting the Results

In [ ]:
h5_data = "acoustic_snapshots.h5"
pestoseis.seismicwaves2d.animatewaves.animateacousticwaves(h5_data, clipamplitude=0.1)